In [ ]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector
from pynput.keyboard import Controller, Key
import time

# Keyboard layout matching the image description
KEYBOARD_LAYOUT = [
    ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "F"],
    ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
    ["Shift", "Z", "X", "C", "V", "B", "N", "M", ",", "."],
    ["Space", "Ent", "Bac", "Clear"]
]


class VirtualAIKeyboard:
    def __init__(self):
        self.cap = cv2.VideoCapture(0)
        self.cap.set(3, 1280)  # Increased width for better text display
        self.cap.set(4, 720)   # Increased height
        self.detector = HandDetector(detectionCon=0.8, maxHands=1)
        self.keyboard = Controller()
        self.text = ""
        self.is_shift = False
        self.suggestions = []
        self.word_database = ["hello", "world", "python", "keyboard", "virtual", "vision", "computer"]
        self.create_keyboard_buttons()
        self.COLORS = {
            "primary": (255, 0, 255),
            "highlight": (0, 255, 0),
            "text": (255, 255, 255),
            "background": (50, 50, 50),
            "shift_active": (0, 200, 255)  # Special color for active shift
        }
        self.last_key_press_time = 0
        self.debounce_time = 0.3  # seconds

    def create_keyboard_buttons(self):
        self.buttons = []
        button_size = 70
        start_x, start_y = 100, 200  # Adjusted for larger text display
        x_gap, y_gap = 10, 10

        for row_idx, row in enumerate(KEYBOARD_LAYOUT):
            for key_idx, key in enumerate(row):
                # Handle special keys with different widths
                if key in ["Space"]:
                    width = button_size * 5
                elif key in ["Ent", "Shift", "Bac", "Clear"]:
                    width = button_size * 2
                else:
                    width = button_size

                pos = (
                    start_x + (button_size + x_gap) * key_idx,
                    start_y + (button_size + y_gap) * row_idx
                )
                self.buttons.append({
                    "key": key,
                    "pos": pos,
                    "size": (width, button_size),
                    "active": False
                })

    def draw_keyboard(self, img):
        # Draw text display area with dynamic height
        text_display_height = max(100, min(400, 50 + len(self.text) // 30 * 20))
        cv2.rectangle(img, (50, 50), (1230, 50 + text_display_height), (100, 100, 100), cv2.FILLED)
        cv2.rectangle(img, (50, 50), (1230, 50 + text_display_height), (200, 200, 200), 2)

        # Draw text with word wrapping
        y_offset = 90
        words = self.text.split(' ')
        line = ""
        for word in words:
            test_line = line + word + " "
            if cv2.getTextSize(test_line, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0][0] < 1150:
                line = test_line
            else:
                cv2.putText(img, line, (60, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                            self.COLORS["text"], 2)
                y_offset += 30
                line = word + " "

        # Draw the last line
        if line:
            cv2.putText(img, line, (60, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        self.COLORS["text"], 2)

        # Draw shift status
        shift_status = "SHIFT: ON" if self.is_shift else "SHIFT: OFF"
        cv2.putText(img, shift_status, (1000, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                    self.COLORS["shift_active"] if self.is_shift else self.COLORS["text"], 2)

        # Draw buttons
        for btn in self.buttons:
            x, y = btn["pos"]
            w, h = btn["size"]

            if btn["key"] == "Shift" and self.is_shift:
                color = self.COLORS["shift_active"]
            else:
                color = self.COLORS["highlight"] if btn["active"] else self.COLORS["primary"]

            cv2.rectangle(img, (x, y), (x + w, y + h), color, cv2.FILLED)
            cv2.rectangle(img, (x, y), (x + w, y + h), (200, 200, 200), 2)

            text_size = cv2.getTextSize(btn["key"], cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
            text_x = x + (w - text_size[0]) // 2
            text_y = y + (h + text_size[1]) // 2
            cv2.putText(img, btn["key"], (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, self.COLORS["text"], 2)

        return img

    def process_key_press(self, key):
        current_time = time.time()
        if current_time - self.last_key_press_time < self.debounce_time:
            return

        self.last_key_press_time = current_time

        if key == "Bac":
            self.text = self.text[:-1] if self.text else ""
            self.keyboard.press(Key.backspace)
            self.keyboard.release(Key.backspace)
        elif key == "Shift":
            self.is_shift = not self.is_shift
        elif key == "Clear":
            self.text = ""
        elif key == "Ent":
            self.keyboard.press(Key.enter)
            self.keyboard.release(Key.enter)
            self.text += "\n"
        elif key == "Space":
            self.keyboard.press(" ")
            self.keyboard.release(" ")
            self.text += " "
        else:
            if key == ";":
                actual_key = ":" if self.is_shift else ";"
            elif key == ",":
                actual_key = "<" if self.is_shift else ","
            elif key == ".":
                actual_key = ">" if self.is_shift else "."
            else:
                actual_key = key.upper() if self.is_shift else key.lower()

            self.keyboard.press(actual_key)
            self.keyboard.release(actual_key)
            self.text += actual_key

            if self.is_shift and key not in ["Shift", "Space", "Ent", "Bac", "Clear"]:
                self.is_shift = False

    def detect_key_press(self, lmList, img):
        if not lmList or len(lmList) < 9:
            return

        x8, y8 = lmList[8][0], lmList[8][1]

        for btn in self.buttons:
            x, y = btn["pos"]
            w, h = btn["size"]
            if x <= x8 <= x + w and y <= y8 <= y + h:
                btn["active"] = True
                x1, y1 = lmList[8][0], lmList[8][1]
                x2, y2 = lmList[4][0], lmList[4][1]
                length = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

                if length < 40:
                    self.process_key_press(btn["key"])
                    cv2.circle(img, (x8, y8), 20, (0, 255, 0), cv2.FILLED)
                    break
            else:
                btn["active"] = False

    def run(self):
        while True:
            success, img = self.cap.read()
            if not success:
                continue

            img = cv2.flip(img, 1)
            hands, img = self.detector.findHands(img, flipType=False)
            lmList = hands[0]['lmList'] if hands else []

            if hands:
                self.detect_key_press(lmList, img)

            img = self.draw_keyboard(img)

            cv2.putText(img, "Pinch to type | Press 'q' to quit", (50, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)

            cv2.imshow("AI Virtual Keyboard", img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cap.release()
        cv2.destroyAllWindows()


if __name__ == "__main__":
    vk = VirtualAIKeyboard()
    vk.run()
